In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/from-hack-true/result.csv
/kaggle/input/from-hack/train_data.csv


In [3]:
import pandas as pd

In [189]:
import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch import nn
from sklearn.metrics import f1_score

In [36]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cpu'

In [20]:
from sklearn.model_selection import train_test_split

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cpu'

In [15]:
df = pd.read_csv("/kaggle/input/from-hack-true/result.csv")
df.head(5)

,Unnamed: 0,Question,Category,answer_class,Qs0,Qs1,Qs2,Qs3,Qs4,Qs5,Qs6
0,0,"Что делать, если я хочу изменить группу или сп...",Перевод/ запись в группу,11,Как перейти в другую группу или сменить специа...,"Какие шаги нужно предпринять, чтобы изменить г...",Как поменять группу или специализацию в процес...,Как осуществить перевод в другую группу или см...,Как изменить группу или направление обучения в...,"Какие действия нужно предпринять, чтобы сменит...",Как перевестись в другую группу или сменить сп...
1,1,Из чего состоит основное обучение в личном каб...,Портал,15,Какие элементы входят в основное обучение в ли...,Какие компоненты составляют основное обучение ...,Какие составляющие входят в основное обучение ...,Какие части составляют основное обучение в лич...,Какие элементы являются частью основного обуче...,Какие компоненты являются основой обучения в л...,Какие составляющие являются основой обучения в...
2,2,Что входит в программу помощи с трудоустройств...,Трудоустройство,27,Какие инструменты и ресурсы предоставляются ст...,Какие услуги и ресурсы доступны студентам в ра...,Какие инструменты и ресурсы включены в програм...,Какие ресурсы и инструменты предоставляются ст...,Какие инструменты и ресурсы предлагаются студе...,Какие ресурсы и инструменты доступны студентам...,Какие инструменты и ресурсы предоставляются ст...
3,3,"Подскажите расписание каникул в 2024, пожалуйста",Расписание,21,Можете ли вы предоставить мне расписание каник...,Какое расписание каникул запланировано на 2024...,"Можете ли вы сообщить мне, когда будут каникул...",Какие даты каникул запланированы на 2024 год?,Можете ли вы предоставить мне информацию о кан...,Когда будут каникулы в 2024 году?,Можете ли вы подсказать мне расписание каникул...
4,4,Какие инструменты и ресурсы вы предоставляете ...,Трудоустройство,25,Какие инструменты и ресурсы доступны для помощ...,Какие ресурсы и инструменты предоставляются дл...,Какие инструменты и ресурсы предлагаются для п...,Какие ресурсы и инструменты доступны для помощ...,Какие инструменты и ресурсы предоставляются дл...,Какие ресурсы и инструменты предлагаются для п...,Какие инструменты и ресурсы доступны для помощ...


In [16]:
df["Category"].value_counts()

Category
Документы                   16
Организация уроков          13
Портал                      12
Трудоустройство              6
Программа обучения           5
Расписание                   4
Требования ПО                4
Перевод/ запись в группу     3
Оценки                       3
Практические работы          3
Name: count, dtype: int64

In [151]:
mean_weight = 552 / 10

In [160]:
cat_weights = torch.tensor([mean_weight / 16, mean_weight / 13, mean_weight / 12, mean_weight / 6, mean_weight / 5, mean_weight / 4, mean_weight / 4, mean_weight / 3, mean_weight / 3, mean_weight / 3])

In [17]:
categories = {"Документы": 0, "Организация уроков": 1, "Портал": 2, "Трудоустройство": 3, "Программа обучения": 4, "Расписание": 5, "Требования ПО": 6, "Перевод/ запись в группу": 7, "Оценки": 8, "Практические работы": 9}

In [18]:
df["Category"] = df["Category"].apply(lambda x: categories[x])
df.head(5)

,Unnamed: 0,Question,Category,answer_class,Qs0,Qs1,Qs2,Qs3,Qs4,Qs5,Qs6
0,0,"Что делать, если я хочу изменить группу или сп...",7,11,Как перейти в другую группу или сменить специа...,"Какие шаги нужно предпринять, чтобы изменить г...",Как поменять группу или специализацию в процес...,Как осуществить перевод в другую группу или см...,Как изменить группу или направление обучения в...,"Какие действия нужно предпринять, чтобы сменит...",Как перевестись в другую группу или сменить сп...
1,1,Из чего состоит основное обучение в личном каб...,2,15,Какие элементы входят в основное обучение в ли...,Какие компоненты составляют основное обучение ...,Какие составляющие входят в основное обучение ...,Какие части составляют основное обучение в лич...,Какие элементы являются частью основного обуче...,Какие компоненты являются основой обучения в л...,Какие составляющие являются основой обучения в...
2,2,Что входит в программу помощи с трудоустройств...,3,27,Какие инструменты и ресурсы предоставляются ст...,Какие услуги и ресурсы доступны студентам в ра...,Какие инструменты и ресурсы включены в програм...,Какие ресурсы и инструменты предоставляются ст...,Какие инструменты и ресурсы предлагаются студе...,Какие ресурсы и инструменты доступны студентам...,Какие инструменты и ресурсы предоставляются ст...
3,3,"Подскажите расписание каникул в 2024, пожалуйста",5,21,Можете ли вы предоставить мне расписание каник...,Какое расписание каникул запланировано на 2024...,"Можете ли вы сообщить мне, когда будут каникул...",Какие даты каникул запланированы на 2024 год?,Можете ли вы предоставить мне информацию о кан...,Когда будут каникулы в 2024 году?,Можете ли вы подсказать мне расписание каникул...
4,4,Какие инструменты и ресурсы вы предоставляете ...,3,25,Какие инструменты и ресурсы доступны для помощ...,Какие ресурсы и инструменты предоставляются дл...,Какие инструменты и ресурсы предлагаются для п...,Какие ресурсы и инструменты доступны для помощ...,Какие инструменты и ресурсы предоставляются дл...,Какие ресурсы и инструменты предлагаются для п...,Какие инструменты и ресурсы доступны для помощ...


In [22]:
df = pd.melt(df, id_vars=["Category", "answer_class"], value_vars=["Qs0", "Qs1", "Qs2", "Qs3", "Qs4", "Qs5", "Qs6", "Question"], var_name="ques", value_name="text")

In [23]:
df.head()

,Category,answer_class,ques,text
0,7,11,Qs0,Как перейти в другую группу или сменить специа...
1,2,15,Qs0,Какие элементы входят в основное обучение в ли...
2,3,27,Qs0,Какие инструменты и ресурсы предоставляются ст...
3,5,21,Qs0,Можете ли вы предоставить мне расписание каник...
4,3,25,Qs0,Какие инструменты и ресурсы доступны для помощ...


In [49]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [53]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [65]:
text = df["text"].values
labels = torch.tensor(df["Category"].values)

In [66]:
features = tokenizer(text.tolist(), padding=True, truncation=True, return_tensors='pt')
features
input_ids = features["input_ids"]
attention_mask = features["attention_mask"]

In [81]:
dataset = TensorDataset(input_ids.to(device), attention_mask.to(device), labels.to(device))

In [154]:
train_dataset, test_dataset = train_test_split(dataset, random_state=42, test_size=0.35)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [84]:
class RubertTinyClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2").to(device)
        for param in self.bert.parameters():
            param.requires_grad = False

        self.pre_trained_output_len = self.bert.encoder.layer[1].output.dense.out_features
        self.linear = torch.nn.Linear(self.pre_trained_output_len, num_classes)
    
    def forward(self, input_ids, attention_mask, token_type_ids=None):
        return self.linear(self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).pooler_output)

In [216]:
model = RubertTinyClassifier(10)
loss_fn = nn.CrossEntropyLoss(cat_weights)
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [217]:
def accuracy_score(y_pred, y_true):
    return (y_pred.argmax(axis=1) == y_true).sum() / len(y_pred)

In [218]:
from torch.nn.functional import softmax

In [224]:

ids, mask, label = next(iter(train_dataloader))
with torch.inference_mode():
    pred = model(ids, mask).argmax(axis=1)

f1_score(pred, label, average="micro")

1.0

In [220]:
def train(model, dataloader, loss_fn, optim, epochs=10):
    os.makedirs('checkpoint', exist_ok=True)
    model.train()
    for epoch in range(epochs):
        mean_loss = 0
        mean_score = 0
        micro = 0
        for input_ids, attention_mask, label in dataloader:
            model.zero_grad()
            
            pred = model(input_ids, attention_mask)
            
            loss = loss_fn(pred, label)
            
            mean_loss += loss.item()
            
            mean_score += accuracy_score(pred, label)
            
            micro += f1_score(pred.argmax(axis=1), label, average="micro")
            
            loss.backward()
            
            optim.step()
        
        torch.save(model.state_dict(), f'checkpoint/BertClassifier{epoch}.pt')
        
        mean_loss /= len(dataloader)
        mean_score /= len(dataloader)
        micro /= len(dataloader)
        mean_score *= 100
        
        test_result = test(model, test_dataloader, loss_fn)
        print(f"Epoch: {epoch} | Train Loss: {mean_loss:.3f} | Train Score: {mean_score:.2f}% | micro-f1-score: {micro:.3f} |" + test_result)
        

In [221]:
train(model, train_dataloader, loss_fn, optim, epochs=60)

Epoch: 0 | Train Loss: 1.960 | Train Score: 33.79% | micro-f1-score: 0.338 || Test Loss: 1.473 | Test Score: 60.10% | micro-f1-score: 0.601
Epoch: 1 | Train Loss: 1.174 | Train Score: 78.08% | micro-f1-score: 0.781 || Test Loss: 1.046 | Test Score: 77.88% | micro-f1-score: 0.779
Epoch: 2 | Train Loss: 0.819 | Train Score: 83.88% | micro-f1-score: 0.839 || Test Loss: 0.780 | Test Score: 82.21% | micro-f1-score: 0.822
Epoch: 3 | Train Loss: 0.617 | Train Score: 86.78% | micro-f1-score: 0.868 || Test Loss: 0.598 | Test Score: 91.35% | micro-f1-score: 0.913
Epoch: 4 | Train Loss: 0.518 | Train Score: 91.58% | micro-f1-score: 0.916 || Test Loss: 0.605 | Test Score: 82.21% | micro-f1-score: 0.822
Epoch: 5 | Train Loss: 0.418 | Train Score: 89.67% | micro-f1-score: 0.897 || Test Loss: 0.431 | Test Score: 92.79% | micro-f1-score: 0.928
Epoch: 6 | Train Loss: 0.343 | Train Score: 93.84% | micro-f1-score: 0.938 || Test Loss: 0.420 | Test Score: 91.83% | micro-f1-score: 0.918
Epoch: 7 | Train Los

In [197]:
def test(model, dataloader, loss_fn):
    model.eval()
    mean_loss = 0
    mean_score = 0
    micro = 0
    with torch.inference_mode():
        for input_ids, attention_mask, label in dataloader:

            pred = model(input_ids, attention_mask)

            loss = loss_fn(pred, label)

            mean_loss += loss.item()

            mean_score += accuracy_score(pred, label)
            
            micro += f1_score(pred.argmax(axis=1), label, average="micro")


        mean_loss /= len(dataloader)
        mean_score /= len(dataloader)
        micro /= len(dataloader)
        mean_score *= 100
        result = f"| Test Loss: {mean_loss:.3f} | Test Score: {mean_score:.2f}% | micro-f1-score: {micro:.3f}"
    model.train()
    return result

In [198]:
test(model, test_dataloader, loss_fn)

'| Test Loss: 0.494 | Test Score: 91.83% | micro-f1-score: 0.918'

In [204]:
from IPython.display import FileLink

In [222]:
FileLink("./checkpoint/BertClassifier52.pt")

/kaggle/working/checkpoint/BertClassifier52.pt

In [206]:
!pwd

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/kaggle/working


In [223]:
model = RubertTinyClassifier(10)
model.load_state_dict(torch.load("/kaggle/working/checkpoint/BertClassifier52.pt"))


<All keys matched successfully>

In [214]:
torch.load("/kaggle/working/checkpoint/BertClassifier6.pt")

RubertTinyClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwi